z = solução otima


In [59]:
!pip install mip

In [278]:
from mip import Model, CBC ,MAXIMIZE,CONTINUOUS, BINARY, xsum, OptimizationStatus
import copy

In [228]:
class Problem:
    def __init__(self, objective_coeffs, constraints_coeffs, constraints_rhs):
        self.objective_coeffs = objective_coeffs #coef das funcoes objetivo.
        self.constraints_coeffs = constraints_coeffs #coefs das restricçoes do problema
        self.constraints_rhs = constraints_rhs # lados direitos das restricoes

    #def __str__(self): #funcao para printar o problema
    #   return f"Maximize {self.objective_coeffs} \nSubject to {self.constraints_coeffs} <= {self.constraints_rhs}"

In [279]:
class No:
    def __init__(self, model):
        self.model = model

    def copy(self):
      return No(copy.deepcopy(self.model))

In [281]:
def ler_dados(nome_arquivo):
    with open(nome_arquivo, 'r') as arq:
        linhas = arq.readlines()

    num_variaveis, num_restricoes = map(int, linhas[0].split())

    # coeficientes da funcao objetivo
    obj_coef = list(map(float, linhas[1].split()))

    restricoes = [] # coeficientes das restricoes
    rhs = [] # right hand side das restricoes

    for i in range(2, num_restricoes+2):
        restricoes.append(list(map(float, linhas[i].split()))[:-1])

    for i in range(2, num_restricoes+2):
        #rhs.append(list(map(float,linhas[i].split()[-2:-1])))
        rhs.append(float(linhas[i].split()[-1]))

    return Problem(obj_coef, restricoes, rhs)

In [282]:
#main_path = '/Users/dorie/Downloads/UFPB/P4/PO/github/PESQUISA_OPERACIONAL/testes_utilizados'
arq1 =  '/content/teste1.txt'
arq2 = '/content/teste2.txt'
arq3 = '/content/teste3.txt'
arq4 = '/content/teste4.txt'
#C:\Users\dorie\Downloads\UFPB\P4\PO\github\Pesquisa_Operacional\testes_utilizados\teste1.txt

In [283]:
def modelo(problema):


    qtd_restricoes = len(problema.constraints_coeffs)
    qtd_variaveis = len(problema.objective_coeffs)



    model = Model(sense=MAXIMIZE, solver_name=CBC)
    #lb e ub são os limites inferiores e superiores das VARIAVEIS e não do problema.
    x = [model.add_var(var_type=CONTINUOUS,name=f"x_{i}",lb=0,ub=1.0) for i in range(len(problema.constraints_coeffs))]


    '''print("Valores das variáveis x antes da otimização:")
    for variavel in x:
        print(f"{variavel.name}: {variavel.x}")'''

    model.objective = xsum(problema.objective_coeffs[i]*x[i] for i in range(qtd_variaveis))

    for i in range(qtd_restricoes):
        exp_restricao = xsum(problema.constraints_coeffs[i][j]*x[j] for j in range(qtd_variaveis))
        model += exp_restricao <= problema.constraints_rhs[i]


    #model.optimize()

    '''print("Status da otimização:", status)
    print("Valor da função objetivo:", model.objective_value)'''

    lista_solucao = []
    #print("Solution:")
    for v in model.vars:
        lista_solucao.append(v.x)

    return model

In [284]:
# identificar branching variable

def identify_branching_variable(node):
    variables = node.model.vars

    list_dif = []
    for var in variables:
        min_dif = (abs(var.x-0.5))
        list_dif.append(min_dif)

    min_dif = min(list_dif)

    for var in variables:
      if var.x == (min_dif+0.5):
        return var

    print('Não há variável fracionária')
    return None

In [285]:
#@title testando
"""
problem1 = ler_dados(arq1)
model_1 = modelo(problem1)
no_test = No(model_1)

a = identify_branching_variable(no_test)
print(a.x)
"""

'\nproblem1 = ler_dados(arq1)\nmodel_1 = modelo(problem1)\nno_test = No(model_1)\n\na = identify_branching_variable(no_test)\nprint(a.x)\n'

In [286]:
# funcao bound

class BranchAndBound():

    def __init__(self):
        self.primal = 0
        self.Z = 0
        self.solution_vars = []

    def bound(self, node):
      node.model.optimize()

      if node.model.status == OptimizationStatus.INFEASIBLE:
        return "INVIAVEL",[], None

      if node.model.status == OptimizationStatus.NO_SOLUTION_FOUND:
        return "NO SOLUTION",[], None

      if node.model.objective_value <= self.primal:
        return "PODADO",[], None

      if not identify_branching_variable(node):
        print('Solução integral encontrada')
        return "INTEGRALIDADE",node.model.vars,node.model.objective_value

      return "FRACIONARIO", node.model.vars, node.model.objective_value


    def branch_and_bound(self,node):
        queue = [node]

        while queue:
            print("whilw")
            status, var_solutions, objective = self.bound(queue[0])

            print(status, var_solutions, objective)

            if status in ['INVIAVEL','NO SOLUTION','PODADO']:
                queue.pop(0)

                print("if1", queue)
                queue

            elif status == 'INTEGRALIDADE':
              if objective > self.primal:
                self.primal = objective
                self.Z = objective
                self.solutions_vars = var_solutions

                print("if2")

            elif status == 'FRACIONARIO':
                no_explorado = queue.pop(0)
                branch_var = identify_branching_variable(no_explorado)

                print("if3", queue)

                # xj = 0
                novo_no1 = no_explorado
                novo_no1.model += branch_var == 0

                # xj = 1
                novo_no2 = no_explorado
                novo_no2.model += branch_var == 1

                queue.append(novo_no1)
                queue.append(novo_no2)

                print("pos fracionario", queue)

            print(status, var_solutions, objective)

        return self.primal, self.Z, self.solution_vars

In [295]:
def bound(node):
  node.model.optimize()

  if node.model.status == OptimizationStatus.INFEASIBLE:
    return "INVIAVEL",[], None

  if node.model.status == OptimizationStatus.NO_SOLUTION_FOUND:
    return "NO SOLUTION",[], None

  if node.model.objective_value <= 0:
    return "PODADO",[], None

  if not identify_branching_variable(node):
    print('Solução integral encontrada')
    return "INTEGRALIDADE",node.model.vars,node.model.objective_value

  return "FRACIONARIO", node.model.vars, node.model.objective_value




problem1 = ler_dados(arq1)
model_1 = modelo(problem1)

no_test = No(model_1)
'''no_test.model.optimize()
no_test.model.objective_value'''

b = bound(no_test)
b[2]



28.749999999999996

In [296]:
queue = [no_test]
print(queue[0].model.objective_value)

status, var_solutions, objective = bound(queue[0])

print(status, var_solutions, objective)

no_explorado = queue.pop(0)

print(no_explorado)
queue

branch_var = identify_branching_variable(no_explorado)

print(branch_var)

# xj = 0
novo_no1 = no_explorado
novo_no1.model += branch_var == 0

queue.append(novo_no1)

print(queue[0].model.objective_value)


28.749999999999996
FRACIONARIO <mip.lists.VarList object at 0x7ed4e41ee110> 28.749999999999996
x_1
28.749999999999996


In [ ]:
problem1 = ler_dados(arq1)
model_1 = modelo(problem1)
no_test = No(model_1)
a = BranchAndBound()
a.branch_and_bound(no_test)

In [ ]:
queue = [1,2,34]
a = queue.pop(0)
print(queue)
print(a)

In [ ]:
class ponto:
    def __init__(self,num,palavra):
        self.num = num
        self.palavra = palavra

    def __copy__(self):
        return (self.num, self.palavra)

p1 = ponto(12,'a')
p2 = p1
p2.num = p1.num + 1
